In [ ]:
import pandas as pd
import numpy as np

# nevermind this part, this is just to display several tables alongside
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [ ]:
# this function will quickly create DataFrames for our toy examples

def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABCD', range(3))

# Concatenation and merging

Let's remind ourselves **3 principles of tidy data**:
- Each variable forms a column
- Each observation forms a row
- Each type of observation forms a separate table

If first 2 priciples are rather easy to digest, the third one at times seems to make life harder rather than easier. In fact, if you split your data into several tables and don't know how to merge it back in a format you need for a certain analysis, you can lose a lot of time. Merging data from different tables can be very daunting if done manually. Data can have diverse type, some rows or columns can be present only in one of the tables, etc. Historically, the problems of merging were addressed by databases, such as SQL. `pandas` provides a lot of functionality in this domain.

Besides that, merging is often necessary when tidying data from different sources, for example, you might have a table for each participant and you want to put them all together.

# Append
There are several distinct types of putting tables together. The easiest one to understand is `append` -- it is a method of `DataFrame` which will take another `DataFrame` and put it directly under the first one, independent of `index` (`index` is preserved from both `DataFrames`).

In [ ]:
df1 = make_df('ABC',range(3))
df1

In [ ]:
# append another copy of the same DataFrame
df1.append()

`append` will try to match columns. If some columns are present in one `DataFrame` but not in another, it will put missing values where appropriate:

In [ ]:
# columns A exists only in df1, column D only in df2
df1 = make_df('ABC',range(3))
df2 = make_df('BCD',range(3))
display('df1','df2','df1.append(df2)')

`append` is very useful when you want to quickly put together some tables with the same type of data, for example tables for separate subjects. But that's just about its functionality.

# Concat
`concat` can be thought of as generalized `append`. It can do all the things `append` can and much more. For example, it can take a list of `DataFrames` and put them all together in `append`-like manner:

In [ ]:
pd.concat([df1, df1, df2, df2])

You can also pass a list of `keys` to `concat` (same size as list of `DataFrames` to concatenate) and each `DataFrame` will have its own key in the index. This is useful in a situation where you merge several subjects and want to keep each one labeled with which subject this came from.

In [ ]:
pd.concat([df1, df1, df2, df2], keys=['df1','df2','df3','df4'])

You can also concatenate along `columns` instead of `index` by passing `axis` argument. In this case `concat` will try to match `index` (as it tried to match columns when you were concatenating along `index`):

In [ ]:
pd.concat([df1, df1, df2, df2], axis='columns')

Let's see a real example of `concat` use. Here I load data from 2 subjects, which is stored in separate *.mat* files. I concatenate them to create a signle table, and I reset the index. Then I name the columns according to the order given me by the person who conducted the experiment. Resulting is one tidy `DataFrame` with 2 subjects.

In [ ]:
from scipy.io import loadmat
s1_mat = loadmat('data/Ale_subj1.mat')
s1_df = pd.DataFrame(s1_mat['Subject1'])
print('Subject1 df shape:', s1_df.shape)

s2_mat = loadmat('data/Ale_subj2.mat')
s2_df = pd.DataFrame(s2_mat['Subject2'])
print('Subject2 df shape:', s2_df.shape)

df_vibr = pd.concat([s1_df, s2_df])
df_vibr = df_vibr.reset_index(drop=True)
df_vibr.columns = ['id_subj','session','trial','s1_int','s1_dur','s1_seed','inter_stim_delay',
              's2_int','s2_dur','s2_seed','pre_stim_delay','task_type','rewarded_choice',
              'subj_choice','s1_motor','s2_motor']
print('Concatenated df shape:',df_vibr.shape)
df_vibr.head()

`concat` is useful for any kind of simple concatenation where we have the same type of data in different tables. However, when we have different *types* of observations in different tables, `concat` will fail us.

# Merge
If merging data sounds confusing to you at any point, it is because it is. There is a whole area of math called *relational algebra*, which creates the theoretical underpinnings of databases and how they work. We won't study any of that, not only it requires a course of its own, but it is also not very frequent for scientists to deal with the kind of data that requires databases.

Merging, however, is important if you want to work with tidy data and avoid data duplication (which is not only inefficient, but also invites errors). So we will learn a bit about that.

In general there are 3 types of joins: one-to-one, many-to-one, and many-to-many. Pandas function `merge()` provides an interface to do all of them, depending on the inputs. The first (one-to-one) referers to the simplest case, when you have 2 sources, and none of them have duplicate entries. In this case joining is usually easy, and basically reminds a concatenation. Let's see it with a toy example:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'department': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

Here the 2 tables have a common column `employee`, but the order is different. We want to merge the two tables consistently, to see the hire date for our different departments. In this case `merge` will automatically find the matching column:

In [ ]:
df3 = pd.merge(df1, df2)
df3

>**Note**: if we tried to perform `concat` here with `axis='columns'`, it would match the `index`, but not the `employee`. You could get around it by first setting the `employee` as the index in both tables, and then perform the `concat` on columns. You could then reset the index and get out the same table. But it is inefficient. Nevertheless, let's do it for the sake of demonstation:

In [ ]:
df1_e = df1.set_index('employee')
df2_e = df2.set_index('employee')
df_e = pd.concat([df1_e, df2_e], axis='columns')
display("df1_e","df2_e","df_e")

One-to-many is when one of your `DataFrames` contains duplicate entries. `merge` will understand that and try to fill in the values appropriately:

In [ ]:
df4 = pd.DataFrame({'department': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3','df4')

In [ ]:
# note how the supervisor column in the resulting DataFrame has Guido
# across from every person in Engineering department
df5 = pd.merge(df3, df4)
df5

Many-to-many is the most confusing type of join, but it is nevertheless well defined mathematically. Consider the following, where we have a `DataFrame` showing one or more skills associated with a particular department. By performing a many-to-many join, we can recover the skills associated with any individual person. Note that some entries in both `df1` and `df6` had to be duplicated; also "R&D" group disappeared in the joined `DataFrame`, because it had no pairings within `df1`.

In [ ]:
df6 = pd.DataFrame({'department': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'R&D'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'science']})
df7 = pd.merge(df1, df6)
display('df1', 'df6', 'df7')

Some things need to be pointed out. 

**First**, when you merge, you can specify a parameter `how`, which can have 1 of 4 values: *left*, *right*, *outer* or *inner*. This controls which values remain in the resulting `DataFrame` if some values are present only in one of the `DataFrames` you're merging. By default `how='inner'`, and it means that the resulting `DataFrame` will have the *intersection* of values from the input `DataFrames`, that is, only values present in both `DataFrames` will be present in the result. That is why we don't have *R&D* in the `df7`: there is no match for it in the `df1`. `outer` is the opposite of `inner` -- all the values will be present in the result. Let's try to do the same merge, but with `how='outer'`:

In [ ]:
df7 = pd.merge(df1, df6, how='outer')
display('df1', 'df6', 'df7')

See how now there is *R&D* in the resulting `DataFrame`, although there is no employee who is in this department.

`left` and `right` just say that values from the first or the second of the input `DataFrames` will be used. In this case, if I used `how='left'`, only values from `df1` would be used, and for `how='right'` -- only from `df6`. (*Left* and *right* just refer to their positions as the inputs to the `merge` function; this terminology, as well as *inner* and *outer*, is taken directly from the database systems, otherwise they might as well be named "first" and "second").

**Second**: `merge` will try to infer which column(s) to use in both `DataFrames` to match the data consistently. However, it is safest to specify it manually, then the outcome is most predictable. If you want to use a certain column, specify `left_on` parameter (for the first input `DataFrame`) and `right_on` (for the second one). This is extremely useful for when you have several columns matching names and they are not consistent with one another, and you want the outcome to be 100% predictable.

In [ ]:
# the result is equivalent to what we had before, but we have more control
df7 = pd.merge(df1, df6, left_on='department', right_on='department')
display('df1', 'df6', 'df7')

Sometimes you want to use index in one of the `DataFrames` for matching. In this case just specify `left_index=True` instead of `left_on` (same for the `right`). 

Let's see it with an example. Before we loaded and concatenated the data from vibration experiment:

In [ ]:
df_vibr.head()

Associated with these data, there is some information about the subjects in the Excel spreadsheet:

> When you try to read the `.xlsx` file, you might get an error "`No module named 'xlrd'`". This is because `pandas` is using another module to load the Excel file, and you need to install that module. Open your computer's terminal (console or command prompt ("cmd") in Windows) and run `pip install xlrd`. Now it should work.

In [ ]:
df_vibr_subj = pd.read_excel('data/Ale_subj_info.xlsx')
df_vibr_subj

If we wanted to look at, for example, performance depending on gender or age, we would have to merge these 2 tables to perform a `groupby`. If we try to do it by hand, it would be very cumbersom. Instead, let's do it using `merge`. First I want to make the column `id_subj` in the `df_vibr_subj` so that it matches the values in the `df_vibr` (i.e. now we have `Sub1`, but we should have just `1`). Then I make it index of the subjects table. I will also drop some other columns so that the result is clearer (otherwise after merge we will have all the columns there and it might be a bit confusing; also, in this example we just need gender and age, so it makes sense to merge in only these columns).

In [ ]:
# rename column
df_vibr_subj.rename(columns={'Subjects':'id_subj'}, inplace=True)
# mapping of old values to new (see Pro-tip below)
subj_mapping_dict = {name_old:int(name_old[-1]) for name_old in df_vibr_subj['id_subj'].unique()}
# replace values based on mapping
df_vibr_subj['id_subj'] = df_vibr_subj['id_subj'].replace(subj_mapping_dict)
# set id_subj as index
df_vibr_subj.set_index('id_subj', inplace=True)
df_vibr_subj = df_vibr_subj[['age','Gender']]
df_vibr_subj

> **Pro-tip**: This line
    
>    `subj_mapping_dict = {name_old:int(name_old[-1]) for name_old in df_vibr_subj['id_subj'].unique()}`
    
>creates a dictionary with mappings for replacing Sub1 with 1, Sub2 with 2, etc; this is called "dict comprehension" and is just an extension of list comprehensions to create a dictionary; go here to learn more: https://www.python.org/dev/peps/pep-0274/

Now that we have the subject's id in both tables, we can merge. I specify `id_subj` as target merging column for `df_vibr` and `index` for `df_vibr_subj`.

In [ ]:
df_result = pd.merge(df_vibr, df_vibr_subj, left_on='id_subj', right_index=True)
df_result.head()

*Voilà!* Now we have age and gender for every trial (last columns) and we could group based on these and calculate some statistics. Note that although in the `df_vibr_subj` we have many more subjects (7 in total), in the `df_vibr` we only have data for 2 subjects. Because the default merge is *inner*, only the data from the 2 subject present in both tables is present in the final table.

Now, for example, we can calculate avegare subject's choice for gender and age (in this case we have only 2 subjects and they are both males, so it doesn't make much sense; but if you had many, it would work flawlessly):

In [ ]:
df_result.groupby(['Gender','age'])['subj_choice'].mean()

# <font color='DarkSeaGreen '>Exercise</font>
Load 2 tables from the data folder: `toy_subjects.csv` and `toy_scores.csv` (use `read_csv` function with `index_col=0` parameter to make the first column into index). In the `toy_scores` calculate the mean `score` for male and female subjects. To do it, you'll need to merge the 2 tables, then group by gender.

# Where to go from here

Here we have only scratched the surface of merging. If you ever need to do complicated joins or are just interested in learning more about joins with `pandas`, I highly recommend <a href="http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.07-Merge-and-Join.ipynb">Combining Datasets: Merge and Join</a> section on the "Python Data Science Handbook" by Jake VanderPlas and <a href="http://pandas.pydata.org/pandas-docs/stable/merging.html">Merge, join, and concatenate</a> section of the `Pandas` documentation.